In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

%matplotlib inline

## Read data files

**Note** We are keeping only the 50 states. DC not included. Self loops not included

In [46]:
def read_US_states(fname):
    states_abb_dict = {}
    states_abb_rev_dict = {}
    states_abb_ord_list = []
    with open(fname, "r") as f:
        for line in f:
            line = line.strip().split(",")
            name = line[0]
            abbr = line[1]
            states_abb_dict[name] = (abbr, len(states_abb_ord_list))
            states_abb_rev_dict[abbr] = name
            states_abb_ord_list.append(abbr)
    
    return states_abb_dict, states_abb_rev_dict, states_abb_ord_list

def read_travel_network(fname, states_abb_dict, states_abb_rev_dict, normalization=1000):
    num_states = 50

    adjacency_list = {}
    A = np.zeros((num_states, num_states))

    for abb in states_abb_rev_dict:
        adjacency_list[abb] = []

    with open(fname, "r") as f:
        for idx, line in enumerate(f):
            if idx == 0:
                continue
            line = line.strip().split(",")
            orig = line[0]
            dest = line[1]
            weight = float(line[2]) / normalization
            if orig == dest:
                continue
            try:
                orig_abb, orig_idx = states_abb_dict[orig]
                dest_abb, dest_idx = states_abb_dict[dest]
                adjacency_list[orig_abb].append((dest_abb, weight))
                A[orig_idx][dest_idx] = weight
            except KeyError:
                pass
    
    return adjacency_list, A

def read_deaths_data(fname):
    df = pd.read_csv(fname)
    sum_df_d = df.groupby(['State', 'Year', 'Quarter'])[['DeathsFromPneumoniaAndInfluenza']].sum().T.to_dict()
    deaths_dict = {}
    for key, deaths in sum_df_d.items():
        state, year, quarter = key
        if state not in deaths_dict:
            deaths_dict[state] = {}
        if year not in deaths_dict[state]:
            deaths_dict[state][year] = {}
        deaths_dict[state][year][quarter] = deaths["DeathsFromPneumoniaAndInfluenza"]
    return deaths_dict

def read_population_dict(fname):
    population_dict = {}
    with open(fname, "r") as f:
        for idx, line in enumerate(f):
            if idx == 0:
                continue
            line = line.strip().split(",")
            state = line[1]
            population_dict[state] = {}
            year = 2009
            for idx, pop in enumerate(line[2:]):
                population_dict[state][year + idx] = int(pop)
    return population_dict

In [50]:
data_dir = "../Data/Clean/"
deaths_fname = "deaths_NCHS_processed.csv"
population_fname = "population.csv"

states_abb_dict, states_abb_rev_dict, stats_abb_ord_list = read_US_states(data_dir + "states_abb.csv")
deaths_dict = read_deaths_data(data_dir + deaths_fname)
population_dict = read_population_dict(data_dir + population_fname)

adj_list = {}
A = {}
A[2009] = {}
A[2009][4] = read_travel_network(data_dir + "2009_Q4.csv", states_abb_dict, states_abb_rev_dict)
network_fname = ["2009_Q4.csv"]
for year in range(2010, 2019):
    A[year] = {}
    for quarter in range(1, 5):
        network_fname = str(year) + "_Q" + str(quarter) + ".csv"
        A[year][quarter] = read_travel_network(data_dir + network_fname, states_abb_dict, states_abb_rev_dict)
A[2019] = {}
A[2019][1] = read_travel_network(data_dir + "2019_Q1.csv", states_abb_dict, states_abb_rev_dict)

In [ ]:
# Air travel the people
# Fraction of infected air travelers is the same as origin state
# Each person has a chance to infect every other person in the new state
# Each of these infected people now have the chance to recover

### Simple 5 x 5 case

Assume population stays same between travels

In [74]:
def travel_and_infect_kernel(A, p_inf, population, infected_fraction, infected_status, verbose=False):
    N = A.shape[0]
    for orig in range(N):
        for dest in range(N):
            if orig == dest:
                continue
            
            # Travel the people
            travel_pop = A[orig, dest]
            infected_pop = int(travel_pop * infected_fraction[orig])
            
            # Infect people at destination
            if infected_pop == 0:
                if verbose:
                    print("{} -> {}: not infecting".format(orig, dest))
                continue
            for person_id, person in enumerate(infected_status[dest]):
                if np.random.binomial(infected_pop, p_inf) > 0:
                    infected_status[dest][person_id] = 1
            
#             # Inject transplants
#             transplant_pop = []
#             for i in range(infected_pop):
#                 transplant_pop.append(1)
#             for i in range(travel_pop - infected_pop):
#                 transplant_pop.append(0)
#             infected_status[dest] = np.concatenate((infected_status[dest], transplant_pop))
        
    return infected_status

def recover_kernel(infected_status, p_recover):
    for state_id, state in enumerate(infected_status):
        for person_id, person in enumerate(state):
            if person == 0:
                continue
            infected_status[state_id][person_id] = np.random.choice([0, 1, 2], p=p_recover)
    return infected_status

def discard_and_inject(infected_status, new_population):
    for state_id, state in enumerate(infected_status):
        state[state == 2] = 0
        difference =  new_population[state_id] - len(state)
        if difference > 0:
            infected_status[state_id] = np.concatenate((state, np.zeros(difference)))
    return infected_status

In [75]:
A = np.array([[0, 1, 2, 3, 4],
              [4, 0, 3, 2, 1],
              [1, 3, 0, 4, 2],
              [2, 4, 1, 0, 3],
              [3, 2, 4, 1, 0]])

pop = [10, 30, 20, 40, 50]
infected = []
for i in pop:
    infected.append(np.zeros(i))
infected[0][8]  = 1
infected[1][1]  = 1
infected[1][27] = 1
infected[2][10] = 1
infected[2][15] = 1
infected[2][16] = 1
infected[4][1]  = 1
infected[4][15] = 1
infected[4][21] = 1
infected[4][37] = 1
infected[3][10] = 1
infected[3][11] = 1
infected[3][12] = 1
infected[3][13] = 1
infected[3][4]  = 1

infected_frac = [np.sum(row) / len(row) for row in infected]
print(infected_frac)

p_inf = 0.001
p_rec = [0.1, 0.8, 0.1]

infected = travel_and_infect_kernel(A, p_inf, pop, infected_frac, infected, verbose=True)
infected = recover_kernel(infected, p_rec)

infected_frac = [len(np.where(row==1)[0]) / (len(row) - len(np.where(row==2)[0])) for row in infected]
infected_dead = [len(np.where(row==2)[0]) for row in infected]
print(infected_frac)
print(infected_dead)

[0.1, 0.06666666666666667, 0.15, 0.125, 0.08]
0 -> 1: not infecting
0 -> 2: not infecting
0 -> 3: not infecting
0 -> 4: not infecting
1 -> 0: not infecting
1 -> 2: not infecting
1 -> 3: not infecting
1 -> 4: not infecting
2 -> 0: not infecting
2 -> 1: not infecting
2 -> 3: not infecting
2 -> 4: not infecting
3 -> 0: not infecting
3 -> 1: not infecting
3 -> 2: not infecting
3 -> 4: not infecting
4 -> 0: not infecting
4 -> 1: not infecting
4 -> 2: not infecting
4 -> 3: not infecting
[0.1, 0.034482758620689655, 0.10526315789473684, 0.1, 0.061224489795918366]
[0, 1, 1, 0, 1]


In [49]:
for i in range(5):
    print(np.where(infected[i]==1))

(array([], dtype=int64),)
(array([], dtype=int64),)
(array([15], dtype=int64),)
(array([4], dtype=int64),)
(array([], dtype=int64),)
